In [2]:
import sys
import numpy as np
sys.path.append("../..")
import random_kingdominion as rm
rm.QUALITIES_AVAILABLE


['village', 'draw', 'thinning', 'gain', 'attack', 'altvp', 'interactivity']

In [3]:
qual_name = "card_amount"



old_df = rm.read_dataframe_from_file(rm.FPATH_CARD_DATA,  eval_lists=True)

fpath = rm.PATH_CARD_INFO.joinpath(f"specifics/{qual_name}.json")
if fpath.is_file():
    old_qualities = rm.read_json_dict_from_file(fpath)

    # Filter for all expansions where the sum of the qualities is still 0:
    sum_of_qual_in_exp = old_df.groupby("Expansion")[qual_name].sum()
    critical_expansions = sum_of_qual_in_exp[sum_of_qual_in_exp.values == 0].keys().to_list()
    print(f"The overall {qual_name} was 0 in the following expansions:\n{critical_expansions}")
    subset = old_df[np.isin(old_df.Expansion, critical_expansions)]
    old_df_qualities = dict(zip(subset["Name"], subset[qual_name])) | old_qualities
else:
    old_df_qualities = {name: 0 for name in old_df.Name}


# Approach: Copy dict, delete all irrelevant lines
temp_fpath = rm.PATH_CARD_INFO.joinpath("specifics/temp_quals/" + qual_name + "_temp.json")
rm.write_dict_to_json_nicely(old_df_qualities, temp_fpath, sort=False)


AttributeError: module 'random_kingdominion' has no attribute 'write_dict_to_json_nicely'

In [11]:
# Copy the new qualities here
new_qualities = rm.read_json_dict_from_file(temp_fpath)
new_qualities = {key: val for key, val in new_qualities.items() if val != 0}
# ! If you want to reset qualities to 0, set them manually here!

old_qualities = rm.read_json_dict_from_file(fpath) if fpath.is_file() else {}
combined_qualities = old_qualities | new_qualities
combined_qualities = {key: val for key, val in combined_qualities.items() if val != 0}
rm.write_dict_to_json_nicely(combined_qualities, fpath)


Writing the file 'C:\Users\fabia\OneDrive\Dokumente\Freizeit\Dominion\Dominion_randomizer\card_info\specifics\gain_quality.json'


In [4]:
# We can now also update the dataframe:
# ! Don't forget to add the new quality to 'qualities_available'!

df = rm.read_dataframe_from_file(rm.FPATH_RAW_DATA)
df = rm.add_split_piles(df)
df = rm.add_info_columns(df)
rm.write_dataframe_to_file(df, rm.FPATH_CARD_DATA)


Overwriting 'C:\Users\fabia\OneDrive\Dokumente\Freizeit\Dominion\Dominion_randomizer\card_info\good_card_data.csv'
Successfully wrote the dominion cards to the file 'C:\Users\fabia\OneDrive\Dokumente\Freizeit\Dominion\Dominion_randomizer\card_info\good_card_data.csv' in the current path.


In [84]:
import pandas as pd
from functools import reduce
def determine_amount(card: pd.Series):
    if "Loot" in card.Types:
        return "2"
    if "State" in card.Types:
        if card.Name == "Lost in the Woods":
            return "1"
        else:
            return "2*"
    iscastle = "Castle" in card.Types and not card.Name == "Castles"
    if iscastle:
        if card.Name in ["Humble Castle", "Small Castle", "Opulent Castle", "King's Castle"]:
            return "1*"
        else:
            return "1"
    isknight = "Knight" in card.Types and not card.Name == "Knights"
    isartifact = "Artifact" in card.Types
    isally = "Ally" in card.Types
    isprize = "Prize" in card.Types
    iszombie = "Zombie" in card.Name
    if any([card.IsLandscape, card.IsOtherThing, isknight, isartifact, isally, isprize, iszombie]):
        return "1"
    if "Ruin" in card.Types:
        return "?"
    if "Heirloom" in card.Types:
        return "2*"
    if "Shelter" in card.Types:
        return "2*"
    special_dict = {"Copper": 46, "Silver": 40, "Gold": 30, "Curse": "10*", 
                    "Platinum": 12, "Ruins": "10*", "Horse": 30, 
                    "Potion": 16, "Teacher": 5, "Peasant": "10*", "Champion": 5,
                    "Will-o'-Wisp": 12, "Wish": 12, "Imp": 12, "Ghost": 5, "Rats": 20,
                    "Ports": 12}
    if card.Name in special_dict:
        return str(special_dict[card.Name])
    if "Traveller" in card.Types:
        return "5"
    if card.Name in rm.ROTATOR_DICT:
        return "4x4"
    if card.Name in reduce(lambda x, y: x+y, rm.ROTATOR_DICT.values()):
        return "4"
    splitpile_cards = reduce(lambda x, y: x+y, [name.split("/") for name in rm.SPLITPILE_DICT.keys()])
    if card.Name in splitpile_cards:
        return "5"
    if card.Name in rm.SPLITPILE_DICT:
        return "2x5"
    if card.Name in reduce(lambda x, y: x+y, rm.ROTATOR_DICT.values()):
        return "4"
    if "Victory" in card.Types:
        return "8*"
    return "10"
df["CardAmount"] = df.apply(determine_amount, axis=1)
# df[["Name", "CardAmount"]].to_csv("test.csv")
rm.write_dataframe_to_file(df, rm.FPATH_CARD_DATA)


Overwriting 'C:\Users\fabia\OneDrive\Dokumente\Freizeit\Dominion\Dominion_randomizer\card_info\good_card_data.csv'
Successfully wrote the dominion cards to the file 'C:\Users\fabia\OneDrive\Dokumente\Freizeit\Dominion\Dominion_randomizer\card_info\good_card_data.csv' in the current path.


In [78]:
df.set_index("Name").loc["Sibyl"]


Expansion                                                           Allies
Types                                                      [Action, Augur]
Cost                                                                    $6
Text                     +4 Cards+1 ActionPut a card from your hand on ...
Actions / Villagers                                                     +1
Cards                                                               +4, -2
Buys                                                                   NaN
Coins / Coffers                                                        NaN
Trash / Return                                                         NaN
Exile                                                                  NaN
Junk                                                                   NaN
Gain                                                                   NaN
Victory Points                                                         NaN
ImagePath                

In [5]:
df = rm.read_dataframe_from_file(rm.FPATH_CARD_DATA, True)
print(df.columns)
md_text = ""
md_text += f"## {qual_name}\n\n"
for qual, strength in enumerate(["Weak", "Medium strength", "Strong"], start=1):
    md_text += f"\n\n## {strength} {qual_name}\n\n- "
    md_text += "\n- ".join(df[df[qual_name] == qual].Name.to_list())
print(md_text)


Index(['Name', 'Expansion', 'Types', 'Cost', 'Text', 'Actions / Villagers',
       'Cards', 'Buys', 'Coins / Coffers', 'Trash / Return', 'Exile', 'Junk',
       'Gain', 'Victory Points', 'ImagePath', 'IsLandscape', 'IsOtherThing',
       'IsInSupply', 'altvp_quality', 'attack_quality', 'draw_quality',
       'gain_quality', 'interactivity_quality', 'thinning_quality',
       'village_quality', 'attack_types', 'thinning_types'],
      dtype='object')
## gain_quality



## Weak gain_quality

- Bureaucrat
- Feast
- Remodel
- Thief
- Bandit
- Mine
- Replace
- Upgrade
- Treasure Map
- Explorer
- Outpost
- Pirate
- Tactician
- Transmute
- Possession
- Talisman
- City
- Mint
- Hoard
- Expand
- Forge
- Remake
- Tournament
- Jester
- Bag of Gold
- Followers
- Trusty Steed
- Duchess
- Tunnel
- Jack of All Trades
- Noble Brigand
- Spice Merchant
- Trader
- Cache
- Ill-Gotten Gains
- Border Village
- Farmland
- Beggar
- Forager
- Hermit
- Death Cart
- Feodum
- Marauder
- Procession
- Bandit Camp
-

In [9]:
from functools import reduce
print(df.columns)
print("**:\n- **".join(np.unique(reduce(lambda x, y: x + y, df["gain_types"].to_list()))))
df[df.thinning_types.apply(lambda x: "Any" in x)]


Index(['Name', 'Expansion', 'Types', 'Cost', 'Text', 'Actions / Villagers',
       'Cards', 'Buys', 'Coins / Coffers', 'Trash / Return', 'Exile', 'Junk',
       'Gain', 'Victory Points', 'ImagePath', 'IsLandscape', 'IsOtherThing',
       'IsInSupply', 'altvp_quality', 'attack_quality', 'draw_quality',
       'gain_quality', 'interactivity_quality', 'thinning_quality',
       'village_quality', 'attack_types', 'gain_types', 'thinning_types'],
      dtype='object')
Boons**:
- **Buys**:
- **Conditional**:
- **Conditional***:
- **Horses**:
- **Loot**:
- **Specific**:
- **Workshop


,Name,Expansion,Types,Cost,Text,Actions / Villagers,Cards,Buys,Coins / Coffers,Trash / Return,...,altvp_quality,attack_quality,draw_quality,gain_quality,interactivity_quality,thinning_quality,village_quality,attack_types,gain_types,thinning_types
1,Chapel,Base,[Action],$2,Trash up to 4 cards from your hand.,NaN,NaN,NaN,NaN,0-4,...,0,0,0,0,0,3,0,[],[],[Any]
16,Remodel,Base,[Action],$4,Trash a card from your hand. Gain a card costi...,NaN,NaN,NaN,NaN,1,...,0,0,0,1,0,2,0,[],"[Workshop, Conditional]","[Any, Remodel]"
28,Sentry,"Base, 2E",[Action],$5,+1 Card+1 ActionLook at the top 2 cards of you...,+1,+1,NaN,NaN,0-2,...,0,0,0,0,0,3,0,[],[],[Any]
44,Masquerade,Intrigue,[Action],$3,+2 CardsEach player with any cards in hand pas...,NaN,+2,NaN,NaN,1?,...,0,1,2,0,3,2,0,[Indirect],[],[Any]
46,Steward,Intrigue,[Action],$3,Choose one: +2 Cards; or +$2; or trash 2 cards...,NaN,+2?,NaN,+$2?,2?,...,0,0,2,0,0,3,0,[],[],[Any]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,Governor,Promo,[Action],$5,+1 ActionChoose one; you get the version in pa...,+1,+3?,NaN,NaN,1?,...,0,0,2,2,3,1,0,[],"[Workshop, Specific, Conditional*]","[Any, Remodel]"
0,Knights,Dark Ages,"[Action, Attack, Knights]",$5*,Shuffle the Knights pile before each game with...,NaN,NaN,NaN,NaN,NaN,...,1,3,0,0,3,1,1,"[Handsize, Trashing, Deck order]",[],[Any]
0,Catapult/Rocks,Empires,"[Action, Attack]",$3,This pile starts the game with 5 copies of Cat...,NaN,NaN,NaN,NaN,NaN,...,1,3,0,1,3,2,0,"[Handsize, Junking, Scoring]","[Specific, Conditional]",[Any]
0,Sauna/Avanto,Promo,[Action],$4,This pile starts the game with 5 copies of Sau...,NaN,NaN,NaN,NaN,NaN,...,0,0,3,0,1,2,1,[],[],[Any]
